# Assignment 2
> **Github repository**: [Assignment 2](https://github.com/josephnguyen0413/02467_Assignment2)
>
> **Group members**:
> - Rune Harlyk (s234814)
> - Joseph Nguyen (s234826)
> - Julius Winkel (s234862)

We have all worked together on the code and answering questions.

## Part 0

In [72]:
import networkx as nx
import json
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm 
import netwulf as nw
import matplotlib.pyplot as plt
import seaborn as sns
import random
from joblib import delayed, Parallel
from collections import defaultdict
from community import community_louvain
from wordcloud import WordCloud
import ast
from functools import lru_cache
from collections import Counter
from scipy.stats import chi2
from nltk.tokenize import MWETokenizer
from itertools import count
# import nltk.book as book
from nltk import PorterStemmer, bigrams as nltk_bigrams
from heapq import nlargest
import nltk
import re
sns.set_theme(style="darkgrid")
from nltk.corpus import stopwords
seed = 42
np.random.seed(seed)
random.seed(seed)

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Rune\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Part 1: Properties of the real-world network of Computational Social Scientists

> * First, calculate the probability (_p_) that makes the expected number of edges in our random network match the actual edge count in the Computational Social Scientists network. Refer to equation 3.2 in your Network Science textbook for guidance. After finding _p_, figure out the average degree (using the given formula). 

$$ p = \frac{2L}{N(N-1)} $$

$$ k = p(N-1) $$

In [58]:
def load_graph(graph_file):
    with open(graph_file, "r") as f:
        data = json.load(f)
    G = nx.readwrite.json_graph.node_link_graph(data)
    return G

graph_file = "network.json"
G = load_graph(graph_file)
N = G.number_of_nodes()
L = G.number_of_edges()
print(f"Number of nodes: {N}, Number of edges: {L}")

In [10]:
def edge_prob(N, L):
    return (2 * L) / (N * (N - 1))

def expected_degree(p, N):
    return p * (N - 1)

p = edge_prob(N, L)
k = expected_degree(p, N)
print(f"Edge probability: {p:.4f}, Expected degree: {k:.4f}")

Edge probability: 0.0006, Expected degree: 8.1563


> * Now, write a function to generate a Random Network that mirrors the Computational Social Scientists network in terms of node count, using your calculated _p_. Generate a random network by linking nodes in every possible pair with probability _p_. **Hint**: you can use the function [``np.random.uniform``](https://numpy.org/doc/stable/reference/random/generated/numpy.random.uniform.html) to draw samples from a uniform probability distribution.   

In [5]:
def random_network(G, seed = 42):
    random.seed(seed)
    N = G.number_of_nodes()
    L = G.number_of_edges()
    p = edge_prob(N, L)
    G_random = nx.erdos_renyi_graph(N, p, seed=seed)
    country_codes = [data['country_code'] for _, data in G.nodes(data=True) if 'country_code' in data]
    nx.set_node_attributes(G_random, {n: {'country_code': random.choice(country_codes)} for n in G_random})
    return G_random

G_random = random_network(G, seed)
print(f"Number of nodes for random network: {G_random.number_of_nodes()}, Number of edges for random network: {G_random.number_of_edges()}")

Number of nodes for random network: 14139, Number of edges for random network: 57596


> * Visualize the network as you did for the Computational Social Scientists network in the exercise above (my version is below). 


In [ ]:
# network, config = nw.visualize(G_random)

# draw links only at first
# nw.draw_netwulf(network)

> * Answer the following questions __(max 200 words in total)__: 
>    - What regime does your random network fall into? Is it above or below the critical threshold?  
>    - According to the textbook, what does the network's structure resemble in this regime?  
>    - Based on your visualizations, identify the key differences between the actual and the random networks. Explain whether these differences are consistent with theoretical expectations.

## TODO Above vizualize and 200 words

In [6]:
L_random = G_random.number_of_edges()
p_random = edge_prob(N, L_random)
p_c = 1 / N
print(f'p: {p_random:.4f}, p_critical: {p_c:.4f}')

p: 0.0006, p_critical: 0.0001


- Since $p \gt p_c$, the random network is a supercritical regime according to section 3.6 in the book "Network Science". That means the random network is above the critical threshold. 
- $\dots$
- One property of real-world networks are that they are heavy tailed distributions. In our case, the top author in the real network had a degree of 339, but for the random network, the top author only has a degree of 22.

> 2. _Shortest Paths_: Here, we will check if the Computational Social Scientists Network exhibits characteristics of a small-world network by analyzing its shortest paths.
> * Begin by identifying the largest connected component within the Computational Social Scientists network. Recall that a connected component is a subset of nodes in which every pair of nodes is connected by a path. For a refresher on connected components, see section 2.9 of the Network Science book. Follow these steps:
>    - Utilize [``nx.algorithms.connected_components``](https://networkx.org/documentation/stable//reference/algorithms/generated/networkx.algorithms.components.connected_components.html) to enumerate all connected components, which will be returned as a list of node subsets.
>    - Select the largest subset of nodes identified in the previous step. Then, create a subgraph of your original network that includes only these nodes, using [``nx.Graph.subgraph``](https://networkx.org/documentation/stable/reference/classes/generated/networkx.Graph.subgraph.html). This subset forms your largest connected component, sometimes referred to as the [giant component](https://en.wikipedia.org/wiki/Giant_component).


In [7]:
cc = nx.algorithms.connected_components(G)
largest_cc = max(cc, key=len)
giant_component = G.subgraph(largest_cc).copy()

> * Calculate the average (unweighted) shortest path length within this giant component by employing [``nx.average_shortest_path_length``](https://networkx.org/documentation/networkx-1.3/reference/generated/networkx.average_shortest_path_length.html).

In [8]:
nx.average_shortest_path_length(giant_component) 

5.168259510783142


> * Perform the same calculation for the giant component of the random network you constructed in the above exercise 1.


In [8]:
N = giant_component.number_of_nodes()
L = giant_component.number_of_edges()
p = edge_prob(N, L)
k = expected_degree(p, N)
print(f"Edge probability: {p:.4f}, Expected degree: {k:.4f}")

Edge probability: 0.0006, Expected degree: 8.3073



> * Reflect on how the average shortest path lengths of the real and random networks compare and answer the following questions, supporting your answers with the theory from the book.
>   - Why do you think I asked you to consider the giant component only?
>   - Why do you think I asked you to consider unweighted edges?
>   - Does the Computational Social Scientists network exhibit the small-world phenomenon?

## Part 2 Network Analysis in Computational Social Science

> __Exercise 1: Mixing Patterns and Assortativity__ 
>


> __Part 1: Assortativity Coefficient__ 
> 1. *Calculate the Assortativity Coefficient* for the network based on the country of each node. Implement the calculation using the formula provided during the lecture, also available in [this paper](https://arxiv.org/pdf/cond-mat/0209450.pdf) (equation 2, here for directed networks). **Do not use the NetworkX implementation.**
>


We use the formula:

$$
r = \frac{\sum_i e_{ii} - \sum_i a_i b_i}{1 - \sum_i a_i b_i}
$$


In [9]:
def assortativity_coefficient(G, attribute):
    mixing_matrix = nx.attribute_mixing_matrix(G, attribute) 

    # Sum by row to get a_i
    ai = np.sum(mixing_matrix, axis=0)

    # sum by column to get b_i
    bi = np.sum(mixing_matrix, axis=1)

    # Diagonal from mixing matrix to get e_ii
    eii = np.trace(mixing_matrix)
    r = (eii - np.sum(ai*bi)) / (1 - np.sum(ai*bi))
    return r

r = assortativity_coefficient(G, 'country_code')
print(f"Assortativity coefficient: {r:.4f}")

Assortativity coefficient: 0.3882


> __Part 2: Configuration model__
> In the following, we are going to assess the significance of the assortativity by comparing the network's assortativity coefficient against that of random networks generated through the configuration model.  
>
> 2. *Implement the configuration model* using the _double edge swap_ algorithm to generate random networks. Ensure each node retains its original degree but with altered connections. Create a function that does that by following these steps:
>   - **a.** Create an exact copy of your original network.
>   - **b.** Select two edges, $e_{1} = (u,v)$ and $e_{2} = (x,y)$, ensuring *u != y* and *v != x*.
>   - **c.** Flip the direction of $e_{1}$ to $e_{1} = (v,u)$ 50% of the time. This ensure that your final results is not biased, in case your edges were sorted (they usually are). 
>   - **d.** Ensure that new edges $e_{1}' = (e_{1}[0],e_{2}[1])$ and $e_{2}' = (e_{2}[0],e_{1}[1])$ do not already exist in the network.
>   - **e.** Remove edges $e_{1}$ and $e_{2}$ and add edges $e_{1}'$ and $e_{2}'$.
>   - **f.** Repeat steps **b** to **e** until you have performed _E*10_ swaps, where E is the total number of edges.


In [10]:
def double_edge_swap(G):
    H = G.copy()
    E = H.number_of_edges()
    total_swaps = E * 10
    edges = list(H.edges())
    swaps = 0
    while swaps < total_swaps:
        i, j = random.sample(range(E), 2)
        u, v = edges[i]
        x, y = edges[j]
        if random.random() < 0.5:
            u, v = v, u
        if u == y or v == x or H.has_edge(u, y) or H.has_edge(x, v):
            continue
        H.remove_edge(u, v)
        H.remove_edge(x, y)
        H.add_edge(u, y)
        H.add_edge(x, v)
        edges[i] = (u, y)
        edges[j] = (x, v)
        swaps += 1
    return H

H = double_edge_swap(G)
assortativity_coefficient(H, 'country_code')

0.0008190055386686307


> 3. *Double check that your algorithm works well*, by showing that the degree of nodes in the original network and the new 'randomized' version of the network are the same.
>
>


In [11]:
orig_degrees = dict(G.degree())
rand_degrees = dict(H.degree())
print("Degrees preserved:", all(orig_degrees[node] == rand_degrees[node] for node in G.nodes()))

Degrees preserved: True


> __Part 3: Analyzing Assortativity in Random Networks__  
>
> 4. *Generate and analyze at least 100 random networks* using the configuration model. For each, calculate the assortativity with respect to the country and plot the distribution of these values. Compare the results with the assortativity of your original network to determine if connections within the same country are significantly higher than chance.
>

Getting assortativity coefficients from 100 random networks

In [12]:
def assortativity_by_country(seed, G):
    G_random = random_network(G, seed=seed)
    return assortativity_coefficient(G_random, 'country_code')

def parallel_r_values(G, n_iters=100, n_jobs=-1):
    parallel_gen = Parallel(return_as="generator", n_jobs=n_jobs)(delayed(assortativity_by_country)(seed, G) for seed in range(n_iters))
    r_values = [r for r in tqdm(parallel_gen, desc="Assortativity for random graphs", total=n_iters)]
    return r_values

r_values = parallel_r_values(G)

Assortativity for random graphs:   0%|          | 0/100 [00:00<?, ?it/s]

Plotting assortativity coefficients

In [13]:
plt.hist(r_values, bins=20, alpha=0.7)
plt.title("Distribution of Assortativity Values")
plt.xlabel("Assortativity")
plt.ylabel("Frequency")
plt.tight_layout()
plt.show()

> __Exercise 2: Zachary's karate club__: In this exercise, we will work on Zarachy's karate club graph (refer to the Introduction of Chapter 9). The dataset is available in NetworkX, by calling the function [karate_club_graph](https://networkx.org/documentation/stable//auto_examples/graph/plot_karate_club.html)
>
> 1. Visualize the graph. Set the color of each node based on the club split (the information is stored as a node attribute). My version of the visualization is below.


In [18]:
G_karate = nx.karate_club_graph()

def plot_graph_split_by_attribute(G, attribute):
    labels = nx.get_node_attributes(G, attribute)
    club_vals = list(set(nx.get_node_attributes(G, attribute).values()))
    colors_list = ['lightblue', 'lightgreen']
    color_map = {club: colors_list[i] for i, club in enumerate(club_vals)}
    colors = [color_map[G.nodes[n][attribute]] for n in G.nodes()]

    nx.draw(G, labels=labels, node_size=1000, node_color=colors)
    plt.show()

plot_graph_split_by_attribute(G_karate, "club")

> 2. Write a function to compute the __modularity__ of a graph partitioning (use **equation 9.12** in the book). The function should take a networkX Graph and a partitioning as inputs and return the modularity.


In [19]:
def community_modularity(G, c, L):
    s = G.subgraph(c)
    L_c = s.number_of_edges()
    k_c = sum(dict(s.degree()).values())
    M_c = L_c / L - (k_c / (2 * L)) ** 2
    return M_c

def modularity(G, partitions):
    L = G.number_of_edges()
    return sum((community_modularity(G, c, L) for c in partitions))

> 3. Explain in your own words the concept of _modularity_.


Modularity is a measuerement that determines how distinctly seperated each community is. If there are few of edges between the communities and many inside each community, then the modularity will be high.

> 4. Compute the modularity of the Karate club split partitioning using the function you just wrote. Note: the Karate club split partitioning is avilable as a [node attribute](https://networkx.org/documentation/networkx-1.10/reference/generated/networkx.classes.function.get_node_attributes.html), called _"club"_.


In [20]:
def partition_by_attribute(G, attribute):
    labels = nx.get_node_attributes(G, attribute)
    groups = defaultdict(set)
    for node, label in labels.items():
        groups[label].add(node)
    return list(groups.values())


karate_communities = partition_by_attribute(G_karate, "club")
karate_modularity = modularity(G_karate, karate_communities) 
print(f"There are {len(karate_communities)} communities")
print(f"The modularity is {karate_modularity}")

There are 2 communities
The modularity is 0.48931623931623935


> 5. Create $1000$ randomized version of the Karate Club network using the _double edge swap_ algorithm you wrote in the exercise above 5. For each of them, compute the modularity of the "club" split and store it in a list.


In [21]:
def random_modularity(G):
    H = double_edge_swap(G)
    communities = partition_by_attribute(H, "club")
    return modularity(H, communities)

def parallel_modularity_karate_networks(G, n_iters=1000, n_jobs=-1):
    parallel_gen = Parallel(return_as="generator", n_jobs=n_jobs)(delayed(random_modularity)(G) for _ in range(n_iters))
    modularity_values = [r for r in tqdm(parallel_gen, desc="Modularity for random graphs", total=n_iters)]
    return modularity_values

modularity_karate_values = parallel_modularity_karate_networks(G_karate)

Modularity for random graphs:   0%|          | 0/1000 [00:00<?, ?it/s]

NameError: name 'double_edge_swap' is not defined

> 6. Compute the average and standard deviation of the modularity for the random network.


In [8]:
modularity_karate_values = np.array(modularity_karate_values)
modularity_karate_avg = modularity_karate_values.mean()
modularity_karate_std = modularity_karate_values.std()
modularity_karate_avg, modularity_karate_std

NameError: name 'modularity_karate_values' is not defined

> 7. Plot the distribution of the "random" modularity. Plot the actual modularity of the club split as a vertical line (use [axvline](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.axvline.html)).


In [31]:
plt.hist(modularity_karate_values, bins=10, alpha=0.7)
plt.axvline(karate_modularity, color='red', linestyle='dashed', linewidth=2)
plt.title("Random Modularity Distribution vs. Karate Club Split")
plt.xlabel("Modularity")
plt.ylabel("Frequency")
plt.tight_layout()
plt.show();

> 8. Comment on the figure. Is the club split a good partitioning? Why do you think I asked you to perform a randomization experiment? What is the reason why we preserved the nodes degree?


> 9.  Use [the Python Louvain-algorithm implementation](https://anaconda.org/auto/python-louvain) to find communities in this graph. Report the value of modularity found by the algorithm. Is it higher or lower than what you found above for the club split? What does this comparison reveal?


In [22]:
def louvain_partitions(G):
    partition = community_louvain.best_partition(G)
    groups = defaultdict(set)
    for node, comm in partition.items():
        groups[comm].add(node)
    return list(groups.values())

karate_louvain_partitions = louvain_partitions(G_karate)
karate_louvain_modularity = modularity(G_karate, karate_louvain_partitions)
print(f"The modularity is {karate_louvain_modularity}  with {len(karate_louvain_partitions)}")

The modularity is 0.5573635765943459  with 4


> 10.  Compare the communities found by the Louvain algorithm with the club split partitioning by creating a matrix **_D_** with dimension (2 times _A_), where _A_ is the number of communities found by Louvain. We set entry _D_(_i_,_j_) to be the number of nodes that community _i_ has in common with group split _j_. The matrix **_D_** is what we call a [**confusion matrix**](https://en.wikipedia.org/wiki/Confusion_matrix). Use the confusion matrix to explain how well the communities you've detected correspond to the club split partitioning.

In [ ]:
def confusion_matrix(partition1, partition2):
    n_cum1, n_cum2 = len(partition1), len(partition2)
    confusion_matrix = np.zeros((n_cum1, n_cum2))
    for i, comunity1 in enumerate(partition1):
        for j, comunity2 in enumerate(partition2):
            confusion_matrix[i, j] = len(set(comunity1) & set(comunity2))

    return confusion_matrix

karate_confusion_matrix = confusion_matrix(karate_communities, karate_louvain_partitions)

sns.heatmap(karate_confusion_matrix, annot=True,
    yticklabels=[f'Club {i+1}' for i in range(len(karate_communities))],
    xticklabels=[f'Louvain {i+1}' for i in range(len(karate_louvain_partitions))]
)
plt.title('Confusion Matrix: Louvain Communities vs Club Split')
plt.xlabel('Club Split')
plt.ylabel('Louvain Communities')
plt.tight_layout()
plt.show()

## Part 3 - Words that characterize Computational Social Science communities

#### Week 6, Exercise 4

In [57]:
G_louvain_partitions = louvain_partitions(G)
author_degrees = dict(G.degree())
author_data = pd.DataFrame({
    "author": list(author_degrees.keys()),
    "degree": list(author_degrees.values()),
    "community": [next((i for i, com in enumerate(G_louvain_partitions) if author in com)) for author in author_degrees.keys()]
})
author_data

author  degree  community
0      A5067008528       6          0
1      A5022754313      25          0
2      A5044056170       4          0
3      A5012701585      17          0
4      A5071496598      15          0
...            ...     ...        ...
14755  A5100431699       4         39
14756  A5101877341       4         39
14757  A5108625853       1         32
14758  A5047601648       1        169
14759  A5087900468       2          6

[14760 rows x 3 columns]

#### Week 7 (tokenized abstract dataframe)

In [67]:
papers_df = pd.read_csv("final_papers.csv")
papers_df['id'] = papers_df['id'].map(lambda x: x[21:])
papers_df['author_ids'] = papers_df['author_ids'].apply(ast.literal_eval)
papers_df['author_ids'] = papers_df['author_ids'].apply(lambda x: [y[21:] for y in x])

abstracts_df = pd.read_csv("final_abstracts.csv")
abstracts_df['id'] = abstracts_df['id'].map(lambda x: x[21:])
abstracts_df["abstract_inverted_index"] = abstracts_df["abstract_inverted_index"].apply(
    lambda x: ast.literal_eval(x) if pd.notna(x) else {}
)
abstracts_df["author_ids"] = abstracts_df["id"].map(papers_df.set_index("id")["author_ids"])
abstracts_df.head()

id                                              title  \
0  W3129422154  Expectations of reward and efficacy guide cogn...   
1  W4392375089  An experimental manipulation of the value of e...   
2  W3087570498  The impact of COVID-19 on the gender division ...   
3  W2905008240  The strength of long-range ties in population-...   
4  W3129992849  Promises and Perils of Experimentation: The Mu...   

                             abstract_inverted_index  \
0  {'The': [0], 'amount': [1], 'of': [2, 26, 62, ...   
1                                                 {}   
2  {'As': [0], 'most': [1, 161], 'other': [2], 'E...   
3  {'The': [0], 'strength': [1], 'of': [2, 80], '...   
4  {'Researchers': [0], 'run': [1], 'experiments'...   

                                          author_ids  
0  [A5067008528, A5022754313, A5044056170, A50127...  
1  [A5022754313, A5010205815, A5062483057, A50127...  
2  [A5069473136, A5045510685, A5084303908, A50519...  
3            [A5034327826, A5054622320, A5017534073]  
4            [A5022754313, A5020488191, A5012701585]

In [15]:
def invert_to_text(inverted_index):
    if not inverted_index:
        return ''
    lenght = max(i for p in inverted_index.values() for i in p) + 1
    text = [''] * lenght
    for word, positions in inverted_index.items():
        for i in positions:
            text[i] = word
    return " ".join(text)

# Apply inverted_index function to df
abstracts_df["text"] = abstracts_df["abstract_inverted_index"].apply(invert_to_text)

# TODO Fix filtering of rows with no data
abstracts_df = abstracts_df[abstracts_df["text"].apply(lambda x: len(x) > 0)]

In [16]:
stemmer = PorterStemmer()
pattern = re.compile(r'https?://\S+|[^a-z\s]')
stop_words = stopwords.words('english')

@lru_cache(maxsize=10000)
def cached_stem(word):
    return stemmer.stem(word)

def tokenize(text, stem=True):
    text = text.lower()
    text = pattern.sub('', text)
    tokens = text.split()
    tokens = [token for token in tokens if token not in stop_words]
    return [stemmer.stem(t) for t in tokens] if stem else tokens

# TODO - skriv om stemming of tokenize strategi


    Revisit Chapter 3 if you're unfamiliar with tokenization.
    Exclude punctuation, URLs, mathematical symbols, and numbers.
    Convert all text to lowercase.
    Optional: Consider using stemming.


In [17]:
abstracts_df["tokens"] = abstracts_df["text"].apply(tokenize)

2954976

## Bigrams

In [18]:
abstracts_df["bigrams"] = abstracts_df["tokens"].apply(lambda x: list(nltk_bigrams(x)))

bigrams = abstracts_df["bigrams"].dropna().explode().tolist()

abstracts_df = abstracts_df.drop('bigrams', axis=1)

bigrams = [bg for bg in bigrams if not isinstance(bg, float)]

In [19]:
bigram_counter = Counter(bigrams)
first_counter = Counter((bg[0] for bg in bigrams))
second_counter = Counter((bg[1] for bg in bigrams))
N = len(bigrams)

def chi_squared(bg):
    w1, w2 = bg
    nii = bigram_counter[bg]
    nio = first_counter[w1] - nii
    noi = second_counter[w2] - nii
    noo = N - (nii + nio + noi)
    
    R1 = first_counter[w1]
    C1 = second_counter[w2]
    R2 = noi + noo
    C2 = nio + noo
    
    E_ii = R1 * C1 / N
    E_io = R1 * C2 / N
    E_oi = R2 * C1 / N
    E_oo = R2 * C2 / N
    
    X_ii = (nii - E_ii)**2 / E_ii
    X_io = (nio - E_io)**2 / E_io
    X_oi = (noi - E_oi)**2 / E_oi
    X_oo = (noo - E_oo)**2 / E_oo
    
    return X_ii + X_io + X_oi + X_oo

chi2vals = {bg: chi_squared(bg) for bg in bigram_counter}
pvals = {bg: chi2.sf(val, 1) for bg, val in chi2vals.items()}
collocs = {bg: cnt for bg, cnt in bigram_counter.items() if cnt > 50 and pvals[bg] < 0.001}

In [20]:
tokenizer = MWETokenizer(list(collocs.keys()), separator="_")
abstracts_df["tokens"] = abstracts_df["tokens"].apply(tokenizer.tokenize)
abstracts_df.to_csv("abstracts_with_tokens.csv", index=False)

In [52]:
final_abstracts_df = pd.read_csv("abstracts_with_tokens.csv")
final_abstracts_df['author_ids'] = final_abstracts_df['author_ids'].apply(ast.literal_eval)
final_abstracts_df['tokens'] = final_abstracts_df['tokens'].apply(ast.literal_eval)
final_abstracts_df.head()

id                                              title  \
0  W3129422154  Expectations of reward and efficacy guide cogn...   
1  W3087570498  The impact of COVID-19 on the gender division ...   
2  W2905008240  The strength of long-range ties in population-...   
3  W3129992849  Promises and Perils of Experimentation: The Mu...   
4  W4386387636  High level of correspondence across different ...   

                             abstract_inverted_index  \
0  {'The': [0], 'amount': [1], 'of': [2, 26, 62, ...   
1  {'As': [0], 'most': [1, 161], 'other': [2], 'E...   
2  {'The': [0], 'strength': [1], 'of': [2, 80], '...   
3  {'Researchers': [0], 'run': [1], 'experiments'...   
4  {'Abstract': [0], 'One': [1], 'widely': [2], '...   

                                          author_ids  \
0  [A5067008528, A5022754313, A5044056170, A50127...   
1  [A5069473136, A5045510685, A5084303908, A50519...   
2            [A5034327826, A5054622320, A5017534073]   
3            [A5022754313, A5020488191, A5012701585]   
4  [A5022754313, A5014437029, A5017914184, A51129...   

                                                text  \
0  The amount of mental effort we invest in a tas...   
1  As most other EU countries, Hungary implemente...   
2  The strength of long-range ties It seems reaso...   
3  Researchers run experiments to test theories, ...   
4  Abstract One widely used approach for quantify...   

                                              tokens  
0  [amount, mental, effort, invest, task, influen...  
1  [eu, countri, hungari, implement, sever, lockd...  
2  [strength, longrang, tie, seem, reason, would,...  
3  [research, run, experi_test, theori, search, d...  
4  [abstract, one, wide_use, approach, quantifi, ...

These exercises are taken from Week 8
> __Exercise 1: TF-IDF and the Computational Social Science communities.__ The goal for this exercise is to find the words charachterizing each of the communities of Computational Social Scientists.
> What you need for this exercise: 
>    * The assignment of each author to their network community, and the degree of each author (Week 6, Exercise 4). This can be stored in a dataframe or in two dictionaries, as you prefer.  
>    * the tokenized _abstract_ dataframe (Week 7, Exercise 2)
>




> 1. First, check out [the wikipedia page for TF-IDF](https://en.wikipedia.org/wiki/Tf%E2%80%93idf). Explain in your own words the point of TF-IDF. 
>   * What does TF stand for? 
>   * What does IDF stand for?


TF stands for term frequency, meaning how frequent a term appears in a document. 

IDF stands for inverse document frequency, which is how unique a term is across all documents. As it is inverse the more frequent the lower the score.


> 2. Now, we want to find out which words are important for each *community*, so we're going to create several ***large documents, one for each community***. Each document includes all the tokens of abstracts written by members of a given community. 
>   * Consider a community _c_
>   * Find all the abstracts of papers written by a member of community _c_.
>   * Create a long array that stores all the abstract tokens 
>   * Repeat for all the communities. 
> __Note:__ Here, to ensure your code is efficient, you shall exploit ``pandas`` builtin functions, such as [``groupby.apply``](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.GroupBy.apply.html) or [``explode``](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.explode.html).


In [56]:
author_counts = author_data["community"].value_counts().reset_index()
author_counts.columns = ["community", "num_authors"]

community_docs = final_abstracts_df.explode("author_ids")
community_docs = community_docs.merge(author_data, left_on="author_ids", right_on="author")
community_docs = community_docs.drop_duplicates(subset=["id", "community"])
community_docs = community_docs.groupby("community")["tokens"].sum().reset_index()

community_docs = community_docs.merge(author_counts, on="community")
community_docs.head()

community                                             tokens  num_authors
0            0  [amount, mental, effort, invest, task, influen...          471
1            1  [eu, countri, hungari, implement, sever, lockd...          162
2            2  [strength, longrang, tie, seem, reason, would,...          439
3            3  [strength, longrang, tie, seem, reason, would,...          609
4            4  [abstract, one, wide_use, approach, quantifi, ...          442
..         ...                                                ...          ...
151        165  [optic, imag, becom_increasingli, promis, real...            8
152        166  [journal_articl, dont, like, facebook, unfrien...           16
153        167  [multipl, factor, play_signific, role, determi...            7
154        169  [preval, diabet, obes, continu, rise, east, as...            8
155        170  [hurrican, lead, substanti, infrastructur, sys...           11

[156 rows x 3 columns]


> 3. Now, we're ready to calculate the TF for each word. Use the method of your choice to find the top 5 terms within the __top 5 communities__ (by number of authors). 


In [ ]:
top5 = community_docs.sort_values("num_authors", ascending=False).head(5)
top5

community                                             tokens  num_authors
15         15  [onlin, harass, complex, grow, problem, twitte...         1005
9           9  [text_gener, larg_languag, model, prolifer, be...          814
12         12  [world, digit, inform, ecosystem, continu, str...          677
6           6  [abstract, one, wide_use, approach, quantifi, ...          619
3           3  [strength, longrang, tie, seem, reason, would,...          609

In [ ]:
def calculate_tf(tokens):
    total = len(tokens)
    counts = Counter(tokens)
    return {word: count / total for word, count in counts.items()}

tf = {}

for _, row in top5.iterrows():
    tf[row["community"]] = calculate_tf(row["tokens"])

top5_per_community = {
  comm: nlargest(5, tf_dict.items(), key=lambda x: x[1])
  for comm, tf_dict in tf.items()
}

for comm, terms in top5_per_community.items():
    print(f"{comm}:")
    for term, score in terms:
        print(f"  {term}: {score}")

15:
  use: 0.007212469164481908
  design: 0.00551259785322109
  user: 0.005029554161734827
  commun: 0.004782501281356357
  task: 0.0042330851742460275
9:
  model: 0.008575665462072684
  use: 0.00687096928502781
  task: 0.0059881801933438575
  gener: 0.004844468217418342
  system: 0.004105186712067248
12:
  network: 0.006623200301476927
  model: 0.00552363548135644
  use: 0.004997083313974798
  user: 0.004904162343260391
  data: 0.003980114912267117
6:
  polit: 0.006517528200843177
  use: 0.005605985795130844
  effect: 0.005195791712560295
  find: 0.004284249306847962
  studi: 0.003813285730563257
3:
  model: 0.006514869661990366
  use: 0.005231391598904982
  popul: 0.005174528646742971
  individu: 0.004922707001454067
  cooper: 0.0046627620772848755


>   * Describe similarities and differences between the communities.
>   * Why aren't the TFs not necessarily a good description of the communities?


We see that many terms appear across multiple of the top 5 communities such as 'use', 'model' and 'user'. However, TF simply computes how often a word appear in each community but does compare the word in relation to other communities. Also we cannot distinguish between communities if a certain word is frequent in all communities. TF-IDF is a better measurement as it downweights more common terms and compares across all communities


>   * Next, we calculate IDF for every word. 
>   * What base logarithm did you use? Is that important?

We will use the natural logarithm. The ranking of the terms will be the same regardless of chosen base logarithm.

In [107]:
def calculate_idf(docs):
    N = len(docs)
    df = defaultdict(int)
    for _, row in docs.iterrows():
        for word in set(row["tokens"]):
            df[word] += 1
    idf = {word: np.log(N / df[word]) for word in df}
    return dict(sorted(idf.items(), key=lambda item: item[1], reverse=True))

idf = calculate_idf(community_docs)
idf

{'stateshappen': 5.049856007249537,
 'nism': 5.049856007249537,
 'monosynapt': 5.049856007249537,
 'povinelli': 5.049856007249537,
 'typetoken': 5.049856007249537,
 'gatedinfer': 5.049856007249537,
 'animalsdescart': 5.049856007249537,
 'shukla': 5.049856007249537,
 'becauseth': 5.049856007249537,
 'theoryneutr': 5.049856007249537,
 'centralitythat': 5.049856007249537,
 'tomgriffithsbrownedu': 5.049856007249537,
 'cherniak': 5.049856007249537,
 'rprincipl': 5.049856007249537,
 'tranform': 5.049856007249537,
 'pur': 5.049856007249537,
 'ellsberg': 5.049856007249537,
 'canini': 5.049856007249537,
 'mindblind': 5.049856007249537,
 'meira': 5.049856007249537,
 'theoryrich': 5.049856007249537,
 'apter': 5.049856007249537,
 'josephwilliamsberkeleyedu': 5.049856007249537,
 'ebert': 5.049856007249537,
 'biguou': 5.049856007249537,
 'hiss': 5.049856007249537,
 'propertiesand': 5.049856007249537,
 'flike': 5.049856007249537,
 'plar': 5.049856007249537,
 'metaparamet': 5.049856007249537,
 'yarden


> 4. We're ready to calculate TF-IDF. Do that for the __top 9 communities__ (by number of authors). Then for each community: 
>   * List the 10 top TF words 
>   * List the 10 top TF-IDF words
>   * List the top 3 authors (by degree)
>   * Are these 10 words more descriptive of the community? If yes, what is it about IDF that makes the words more informative?



In [109]:
top9 = community_docs.sort_values("num_authors", ascending=False).head(9)

In [115]:
def calculate_tf_idf(tf, idf):
    tf_idf = {}
    for word, val in tf.items():
        tf_idf[word] = val * idf[word]
    return tf_idf

tf_idf = {}

for _, row in top9.iterrows():
    tf = calculate_tf(row["tokens"])
    tf_idf[row["community"]] = calculate_tf_idf(tf, idf)

top10_tf_idf_per_community = {
  comm: nlargest(10, tfidf.items(), key=lambda x: x[1])
  for comm, tfidf in tf_idf.items()
}

top10_tf_idf_per_community

{15: [('user', 0.003888800420175994),
  ('design', 0.0035453543224224435),
  ('task', 0.003453137125575927),
  ('commun', 0.003075802378716996),
  ('peopl', 0.0029025770578790027),
  ('particip', 0.0026115404581240546),
  ('visual', 0.0023061497989112804),
  ('work', 0.0023031908144762725),
  ('interact', 0.0020566275863785756),
  ('onlin', 0.001985253051289296)],
 9: [('task', 0.0048848550145124715),
  ('model', 0.0033950687131085525),
  ('text', 0.0031231491908198013),
  ('dataset', 0.0029091735097964496),
  ('learn', 0.002800216278461329),
  ('word', 0.0026929788099226246),
  ('annot', 0.0023557214204023695),
  ('languag', 0.002322369918139267),
  ('semant', 0.002311328194471551),
  ('train', 0.0022766839525365274)],
 12: [('network', 0.004340890863578681),
  ('user', 0.003791848734064351),
  ('dynam', 0.002280874046191479),
  ('model', 0.002186783298428369),
  ('twitter', 0.002015160389279368),
  ('commun', 0.0019588292390872964),
  ('individu', 0.0018875467756503794),
  ('node', 0


 __Exercise 2: The Wordcloud__. It's time to visualize our results!

> * Install the [`WordCloud`](https://pypi.org/project/wordcloud/) module. 
> * Now, create word-cloud for each community. Feel free to make it as fancy or non-fancy as you like.
> * Make sure that, together with the word cloud, you print the names of the top three authors in each community (see my plot above for inspiration). 
> * Comment on your results. What can you conclude on the different sub-communities in Computational Social Science? 
> * Look up online the top author in each community. In light of your search, do your results make sense?

 

__Exercise 3: Computational Social Science__ 

> * Go back to Week 1, Exercise 1. Revise what you wrote on the topics in Computational Social Science. 
> * In light of your data-driven analysis, has your understanding of the field changed? How? __(max 150 words)__